#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time

from arena_api.callback import callback, callback_function
from arena_api.system import system

#### Callbacks: On Event
>    This example demonstrates configuring a callback with events. Events are a
    subset of nodes that invoke callbacks through the underlying events engine.
    The events engine is first initialized to listen for events, then the
    callback is registered using the timestamp test event node. The event is
    generated, along with any data generated from the event. The example then
    waits for the event to process in order to invoke the callback. Registered
    callbacks must also be deregistered before deinitializing the events engine
    in order to avoid memory leaks.


In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


##### Must have the decorator on the callback function
> node.on_update requires node as its first parameter<br>
This function is triggered when the callback event is triggered

In [ ]:
@callback_function.node.on_update
def print_node_value(node, *args, **kwargs):

    print(f'Message from callback')
    print(f'\'{node.name}\' event has triggered this callback')


##### Store initial values
> These initial values are restored to the device after the example is completed

In [ ]:
streamBufferHandlingMode_initial = \
    device.tl_stream_nodemap['StreamBufferHandlingMode'].value
triggerSource_initial = device.nodemap['TriggerSource'].value
triggerSelector_initial = device.nodemap['TriggerSelector'].value
triggerMode_initial = device.nodemap['TriggerMode'].value


##### Register the callback on event node

In [ ]:
event_node = device.nodemap.get_node('EventExposureEnd')

handle = callback.register(event_node, print_node_value)
print(f"Registered '{print_node_value.__name__}' function "
      f"in '{event_node.name}' node")

##### Enable trigger mode before setting the source and selector and before starting the stream. 
>   Trigger mode cannot be turned
    on and off while the device is streaming.<br>
    Make sure Trigger Mode set to 'Off' after finishing this example

In [ ]:
device.nodemap.get_node('TriggerMode').value = 'On'

##### Set the trigger source to software in order to trigger buffers without the use of any additional hardware.
>  Lines of the GPIO can also be used to trigger

In [ ]:
device.nodemap.get_node('TriggerSource').value = 'Software'

device.nodemap.get_node('TriggerSelector').value = 'FrameStart'

device.tl_stream_nodemap.get_node(
    'StreamBufferHandlingMode').value = 'OldestFirst'

##### Initialize events
>    Turn event notification on
    Select the event type to be notified about


In [ ]:
device.initialize_events()
device.nodemap.get_node('EventSelector').value = 'ExposureEnd'
device.nodemap.get_node('EventNotification').value = 'On'


##### Register callback function on event node
>      Allows for manual triggering

In [ ]:
event_node = device.nodemap.get_node('EventExposureEnd')

handle = callback.register(event_node, print_node_value)
print(f"Registered '{print_node_value.__name__}' function "
      f"in '{event_node.name}' node")

##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
> - Continually check until trigger is armed. Once the trigger is armed, it is ready to be executed.
>  Trigger an image buffer manually, since trigger mode is enabled.
> > -    This triggers the camera to acquire a single image buffer.
> > -    A buffer is then filled and moved to the output queue, where it will wait to be retrieved.
> > -    Before the image buffer is sent, the exposure end event will occur. This will happen on every iteration
> - Wait on the event to process it, invoking the registered callback.
> >  -   The data is created from the event generation, not from waiting on it. 
> > -    If the exposure time is long a Timeout exception may occur unless large timeout value is passed to the 'Device.wait_on_event()'
   

In [ ]:
device.start_stream()
for _ in range(10):

    while not device.nodemap.get_node('TriggerArmed').value:
        continue
        
    device.nodemap.get_node('TriggerSoftware').execute()
    
    device.wait_on_event()

##### Deregister each handle in the handle list
> Must be called before device is destroyed

In [ ]:
callback.deregister(handle)
device.deinitialize_events()

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
device.stop_stream()

device.nodemap['TriggerSource'].value = triggerSource_initial
device.nodemap['TriggerSelector'].value = triggerSelector_initial
device.nodemap['TriggerMode'].value = triggerMode_initial

system.destroy_device()
print('Destroyed all created devices')